# Inicjalizacja

## Inicjalizacja zmiennych

In [3]:
tfrecord_path_train_mal = 'drive/My Drive/Data/im_mal_dataset_train.tfrecords'
tfrecord_path_test_mal = 'drive/My Drive/Data/im_mal_dataset_test.tfrecords'

tfrecord_path_train_ben = 'drive/My Drive/Data/im_ben_dataset_train.tfrecords'
tfrecord_path_test_ben = 'drive/My Drive/Data/im_ben_dataset_test.tfrecords'

model_mal_8_path = 'drive/My Drive/Data/model_mal_8.h5'
model_mal_16_path = 'drive/My Drive/Data/model_mal_16.h5'
model_mal_32_path = 'drive/My Drive/Data/model_mal_32.h5'

model_ben_8_path = 'drive/My Drive/Data/model_ben_8.h5'
model_ben_16_path = 'drive/My Drive/Data/model_ben_16.h5'
model_ben_32_path = 'drive/My Drive/Data/model_ben_32.h5'

pickle_path_stats_train_mal_8_mal = 'drive/My Drive/Data/stats_train_mal_8_mal.pickle'
pickle_path_stats_train_mal_16_mal = 'drive/My Drive/Data/stats_train_mal_16_mal.pickle'
pickle_path_stats_train_mal_32_mal = 'drive/My Drive/Data/stats_train_mal_32_mal.pickle'
pickle_path_stats_test_mal_8_mal = 'drive/My Drive/Data/stats_test_mal_8_mal.pickle'
pickle_path_stats_test_mal_16_mal = 'drive/My Drive/Data/stats_test_mal_16_mal.pickle'
pickle_path_stats_test_mal_32_mal = 'drive/My Drive/Data/stats_test_mal_32_mal.pickle'

pickle_path_stats_train_mal_8_ben = 'drive/My Drive/Data/stats_train_mal_8_ben.pickle'
pickle_path_stats_train_mal_16_ben = 'drive/My Drive/Data/stats_train_mal_16_ben.pickle'
pickle_path_stats_train_mal_32_ben = 'drive/My Drive/Data/stats_train_mal_32_ben.pickle'
pickle_path_stats_test_mal_8_ben = 'drive/My Drive/Data/stats_test_mal_8_ben.pickle'
pickle_path_stats_test_mal_16_ben = 'drive/My Drive/Data/stats_test_mal_16_ben.pickle'
pickle_path_stats_test_mal_32_ben = 'drive/My Drive/Data/stats_test_mal_32_ben.pickle'

pickle_path_stats_train_ben_8_mal = 'drive/My Drive/Data/stats_train_ben_8_mal.pickle'
pickle_path_stats_train_ben_16_mal = 'drive/My Drive/Data/stats_train_ben_16_mal.pickle'
pickle_path_stats_train_ben_32_mal = 'drive/My Drive/Data/stats_train_ben_32_mal.pickle'
pickle_path_stats_test_ben_8_mal = 'drive/My Drive/Data/stats_test_ben_8_mal.pickle'
pickle_path_stats_test_ben_16_mal = 'drive/My Drive/Data/stats_test_ben_16_mal.pickle'
pickle_path_stats_test_ben_32_mal = 'drive/My Drive/Data/stats_test_ben_32_mal.pickle'

pickle_path_stats_train_ben_8_ben = 'drive/My Drive/Data/stats_train_ben_8_ben.pickle'
pickle_path_stats_train_ben_16_ben = 'drive/My Drive/Data/stats_train_ben_16_ben.pickle'
pickle_path_stats_train_ben_32_ben = 'drive/My Drive/Data/stats_train_ben_32_ben.pickle'
pickle_path_stats_test_ben_8_ben = 'drive/My Drive/Data/stats_test_ben_8_ben.pickle'
pickle_path_stats_test_ben_16_ben = 'drive/My Drive/Data/stats_test_ben_16_ben.pickle'
pickle_path_stats_test_ben_32_ben = 'drive/My Drive/Data/stats_test_ben_32_ben.pickle'

generate_residuum_mal = True
generate_residuum_ben = True

batch_size = 10
patch_size = 256
num_channels_input = 3
num_channels_target = 3

## Inicjalizacja bibliotek

In [4]:
from google.colab import drive
import tensorflow as tf
import keras
import numpy as np
from keras import layers
import scipy
import pickle
drive.mount('/content/drive')

Mounted at /content/drive


## Funkcje

In [5]:
def parse_example_function(example_proto, image_feature_description):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

def img_img_parse_feature_function(img_features, patch_size, num_channels_input, num_channels_target):
    input_tensor = tf.io.parse_tensor(img_features['input_image_raw'], out_type=tf.float32)
    target_tensor = tf.io.parse_tensor(img_features['target_image_raw'], out_type=tf.float32)   
    input_tensor.set_shape([patch_size, patch_size, num_channels_input])
    target_tensor.set_shape([patch_size, patch_size, num_channels_target])
    return input_tensor, target_tensor

def tfrecord_2_img_img_dataset(tfrecord_path, batch_size, patch_size, num_channels_input=3, num_channels_target=1, num_prefatch=1, num_repeats=1):
    
    image_feature_description = {
        'input_image_raw': tf.io.FixedLenFeature([], tf.string),
        'target_image_raw': tf.io.FixedLenFeature([], tf.string)
    }
    
    _parse_example = lambda example: parse_example_function(example, image_feature_description)
    _parse_feature = lambda feature: img_img_parse_feature_function(feature, patch_size, num_channels_input, num_channels_target)

    dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = dataset.map(_parse_example)
    dataset = dataset.map(_parse_feature)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(num_prefatch)
    dataset = dataset.repeat(num_repeats)
    return dataset

In [6]:
def get_residuum(dset, model):
  output = []

  for elem_in, _ in dset:
    decoded_imgs = model.predict(elem_in)
    output.extend(decoded_imgs - elem_in.numpy())
    
  residuum = np.array(output)

  return residuum

def get_stats(array):
  flatten_array = array.flatten()

  s_max = max(flatten_array)
  s_min = min(flatten_array)
  s_energy = sum(flatten_array)
  s_10p = np.percentile(flatten_array,10)
  s_90p = np.percentile(flatten_array,90)

  s_median = np.median(flatten_array)
  s_mean = np.mean(flatten_array)
  s_range = np.ptp(flatten_array)
  s_var = np.var(flatten_array)
  s_iqr = scipy.stats.iqr(flatten_array)

  s_skew = scipy.stats.skew(flatten_array)
  s_kurtosis = scipy.stats.kurtosis(flatten_array)
  s_medabsdev = scipy.stats.median_absolute_deviation(flatten_array)
  s_meanabsdev = np.mean(np.absolute(flatten_array - np.mean(flatten_array)))
  #s_entropy = scipy.stats.entropy(flatten_array)

  stats = np.array([s_max, s_min, s_energy, s_10p, s_90p,
                    s_median, s_mean, s_range, s_var, s_iqr,
                    s_skew, s_kurtosis, s_medabsdev, s_meanabsdev])
  
  return stats

def get_stats_rgb(array):
  stats = np.array([])
  for i in range(0,3):
    stats = np.append(stats, get_stats(array[:,:,i]))
  
  return stats

def get_stats_all(dset, model):
  resid = get_residuum(dset, model)
  stats = []
 
  for res in resid:
    stats.append(get_stats_rgb(res))

  return np.array(stats)

# Ładowanie danych

## Ładowanie modeli

In [7]:
autoencoder_model_mal_8 = keras.models.load_model(model_mal_8_path)
autoencoder_model_mal_16 = keras.models.load_model(model_mal_16_path)
autoencoder_model_mal_32 = keras.models.load_model(model_mal_32_path)
autoencoder_model_ben_8 = keras.models.load_model(model_ben_8_path)
autoencoder_model_ben_16 = keras.models.load_model(model_ben_16_path)
autoencoder_model_ben_32 = keras.models.load_model(model_ben_32_path)

## Ładowanie TFRecords

In [8]:
dset_train_ben = tfrecord_2_img_img_dataset(tfrecord_path_train_ben, batch_size, patch_size, num_channels_input, num_channels_target, num_repeats = 1)
dset_test_ben = tfrecord_2_img_img_dataset(tfrecord_path_test_ben, batch_size, patch_size, num_channels_input, num_channels_target, num_repeats = 1)

dset_train_mal = tfrecord_2_img_img_dataset(tfrecord_path_train_mal, batch_size, patch_size, num_channels_input, num_channels_target, num_repeats = 1)
dset_test_mal = tfrecord_2_img_img_dataset(tfrecord_path_test_mal, batch_size, patch_size, num_channels_input, num_channels_target, num_repeats = 1)

# Generacja residuum

## Malignant

In [9]:
if generate_residuum_mal:
  stats_stats_train_mal_8_mal = get_stats_all(dset_train_mal, autoencoder_model_mal_8)

  with open(pickle_path_stats_train_mal_8_mal, 'wb') as handle:
    pickle.dump(stats_stats_train_mal_8_mal, handle)

  stats_stats_train_mal_16_mal = get_stats_all(dset_train_mal, autoencoder_model_mal_16)

  with open(pickle_path_stats_train_mal_16_mal, 'wb') as handle:
    pickle.dump(stats_stats_train_mal_16_mal, handle)

  stats_stats_train_mal_32_mal = get_stats_all(dset_train_mal, autoencoder_model_mal_32)

  with open(pickle_path_stats_train_mal_32_mal, 'wb') as handle:
    pickle.dump(stats_stats_train_mal_32_mal, handle)

  stats_stats_test_mal_8_mal = get_stats_all(dset_test_mal, autoencoder_model_mal_8)

  with open(pickle_path_stats_test_mal_8_mal, 'wb') as handle:
    pickle.dump(stats_stats_test_mal_8_mal, handle)

  stats_stats_test_mal_16_mal = get_stats_all(dset_test_mal, autoencoder_model_mal_16)

  with open(pickle_path_stats_test_mal_16_mal, 'wb') as handle:
    pickle.dump(stats_stats_test_mal_16_mal, handle)

  stats_stats_test_mal_32_mal = get_stats_all(dset_test_mal, autoencoder_model_mal_32)

  with open(pickle_path_stats_test_mal_32_mal, 'wb') as handle:
    pickle.dump(stats_stats_test_mal_32_mal, handle)

  ############################################################################

  stats_stats_train_mal_8_ben = get_stats_all(dset_train_ben, autoencoder_model_mal_8)

  with open(pickle_path_stats_train_mal_8_ben, 'wb') as handle:
    pickle.dump(stats_stats_train_mal_8_ben, handle)

  stats_stats_train_mal_16_ben = get_stats_all(dset_train_ben, autoencoder_model_mal_16)

  with open(pickle_path_stats_train_mal_16_ben, 'wb') as handle:
    pickle.dump(stats_stats_train_mal_16_ben, handle)

  stats_stats_train_mal_32_ben = get_stats_all(dset_train_ben, autoencoder_model_mal_32)

  with open(pickle_path_stats_train_mal_32_ben, 'wb') as handle:
    pickle.dump(stats_stats_train_mal_32_ben, handle)

  stats_stats_test_mal_8_ben = get_stats_all(dset_test_ben, autoencoder_model_mal_8)

  with open(pickle_path_stats_test_mal_8_ben, 'wb') as handle:
    pickle.dump(stats_stats_test_mal_8_ben, handle)

  stats_stats_test_mal_16_ben = get_stats_all(dset_test_ben, autoencoder_model_mal_16)

  with open(pickle_path_stats_test_mal_16_ben, 'wb') as handle:
    pickle.dump(stats_stats_test_mal_16_ben, handle)

  stats_stats_test_mal_32_ben = get_stats_all(dset_test_ben, autoencoder_model_mal_32)

  with open(pickle_path_stats_test_mal_32_ben, 'wb') as handle:
    pickle.dump(stats_stats_test_mal_32_ben, handle)

## Benign

In [10]:
if generate_residuum_ben:
  stats_stats_train_ben_8_mal = get_stats_all(dset_train_mal, autoencoder_model_ben_8)

  with open(pickle_path_stats_train_ben_8_mal, 'wb') as handle:
    pickle.dump(stats_stats_train_ben_8_mal, handle)

  stats_stats_train_ben_16_mal = get_stats_all(dset_train_mal, autoencoder_model_ben_16)

  with open(pickle_path_stats_train_ben_16_mal, 'wb') as handle:
    pickle.dump(stats_stats_train_ben_16_mal, handle)

  stats_stats_train_ben_32_mal = get_stats_all(dset_train_mal, autoencoder_model_ben_32)

  with open(pickle_path_stats_train_ben_32_mal, 'wb') as handle:
    pickle.dump(stats_stats_train_ben_32_mal, handle)

  stats_stats_test_ben_8_mal = get_stats_all(dset_test_mal, autoencoder_model_ben_8)

  with open(pickle_path_stats_test_ben_8_mal, 'wb') as handle:
    pickle.dump(stats_stats_test_ben_8_mal, handle)

  stats_stats_test_ben_16_mal = get_stats_all(dset_test_mal, autoencoder_model_ben_16)

  with open(pickle_path_stats_test_ben_16_mal, 'wb') as handle:
    pickle.dump(stats_stats_test_ben_16_mal, handle)

  stats_stats_test_ben_32_mal = get_stats_all(dset_test_mal, autoencoder_model_ben_32)

  with open(pickle_path_stats_test_ben_32_mal, 'wb') as handle:
    pickle.dump(stats_stats_test_ben_32_mal, handle)

  ############################################################################

  stats_stats_train_ben_8_ben = get_stats_all(dset_train_ben, autoencoder_model_ben_8)

  with open(pickle_path_stats_train_ben_8_ben, 'wb') as handle:
    pickle.dump(stats_stats_train_ben_8_ben, handle)

  stats_stats_train_ben_16_ben = get_stats_all(dset_train_ben, autoencoder_model_ben_16)

  with open(pickle_path_stats_train_ben_16_ben, 'wb') as handle:
    pickle.dump(stats_stats_train_ben_16_ben, handle)

  stats_stats_train_ben_32_ben = get_stats_all(dset_train_ben, autoencoder_model_ben_32)

  with open(pickle_path_stats_train_ben_32_ben, 'wb') as handle:
    pickle.dump(stats_stats_train_ben_32_ben, handle)

  stats_stats_test_ben_8_ben = get_stats_all(dset_test_ben, autoencoder_model_ben_8)

  with open(pickle_path_stats_test_ben_8_ben, 'wb') as handle:
    pickle.dump(stats_stats_test_ben_8_ben, handle)

  stats_stats_test_ben_16_ben = get_stats_all(dset_test_ben, autoencoder_model_ben_16)

  with open(pickle_path_stats_test_ben_16_ben, 'wb') as handle:
    pickle.dump(stats_stats_test_ben_16_ben, handle)

  stats_stats_test_ben_32_ben = get_stats_all(dset_test_ben, autoencoder_model_ben_32)

  with open(pickle_path_stats_test_ben_32_ben, 'wb') as handle:
    pickle.dump(stats_stats_test_ben_32_ben, handle)